In [4]:
import os
from urllib.parse import quote
from IPython.display import display, clear_output
import time

import pandas as pd
from datetime import datetime, timedelta, timezone

from sqlalchemy import MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry

from dotenv import load_dotenv
from sqlalchemy import create_engine

def get_engine():
    load_dotenv()
    SERVER = os.getenv("SQL_SERVER_IP")
    PORT= os.getenv("SQL_SERVER_PORT")
    DATABASE = os.getenv("DB")
    USER = os.getenv("DB_USER")
    PASSWORD = os.getenv("DB_PW")
    DRIVER = os.getenv("DB_DRIVER")
    engine = create_engine(
        f"mssql+pyodbc://{USER}:{PASSWORD}@{SERVER}:{PORT}/{DATABASE}?driver={DRIVER}&TrustServerCertificate=yes")
    return engine

def create_object(context, table_name, with_commit=False, **kwargs):
    mapped_class = context["base"].classes[table_name]
    obj = mapped_class()
    for key, value in kwargs.items():
        setattr(obj, key, value)
    context["session"].add(obj)
    if (with_commit):
        context["session"].commit()
    return obj
    
def manipulate_date(input_date, hours=0, minutes=0):
    delta = timedelta(hours=hours, minutes=minutes)    
    manipulated_datetime = input_date + delta
    return manipulated_datetime

def get_time(stamp):
    return datetime.strptime(str(stamp), "%Y-%m-%d %H:%M:%S")

def update_output(text):
    clear_output(wait=True)
    display(text)

df_time_clone = []   
hours_of_day = 24
minutes_of_day = 60   
start_date = datetime(2016, 1, 1, 0, 0, 0)
end_date = datetime(2017, 12, 31, 23, 59, 0)

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)    
mapper_registry = registry()
start = time.time()

df_time = pd.read_sql_table("DIM_CALENDAR", engine.connect())
for index, row in df_time.iterrows():
        progress = "{:.4f}".format(index/len(df_time.index))
        year = str(get_time(row["CAL_DATE"]).year)
        stop= time.time()
        ellapsed = stop - start
        remaining = 0
        if(index!=0):
            remaining = ellapsed/index/len(df_time.index)
        update_output("Year: " + year + " p: " + progress +  " remaining: " + str(remaining))
        now = get_time(row["CAL_DATE"])
        if(now > start_date and now < end_date):
            for h in range(hours_of_day):
                for m in range(minutes_of_day):
                    row_cpy = row.copy()        
                    row_cpy["CAL_DATE"] =  manipulate_date(now, h, m)            
                    df_time_clone.append(row_cpy)#df_time_clone = pd.concat([df_time_clone, row_cpy], axis=0

time_df = pd.DataFrame(df_time_clone)
time_df.sort_values(by="CAL_DATE", inplace=True, ignore_index=True)
time_df.reset_index(inplace=True, drop=True)

'Year: 2025 p: 0.9999 remaining: 4.577264943464767e-07'

In [5]:
time_df.to_sql(name='DIM_CALENDAR_EXTENDED', con=engine, index=False, if_exists='replace')

8